# $$Regressão~Linear: Variáveis~Categóricas(Dummies)$$

### $$Conceito:$$


A **Regressão Linear Simples** é modelo mais simples de regressão linear. Nela, utilizamos apenas uma variável de desfecho e uma variável preditora. Quando utilizamos mais variáveis preditoras, chamamos de Regressão Linear Múltipla.


Na modelagem é comum encontrarmos diversos tipos de modelos que têm como pré-requisito que todas as variáveis sejam numéricas, mas o que fazer com as **variáveis categóricas** nessas situações?



### $$Variáveis~Dummies$$

Uma forma de introduzir **características qualitativas** em modelos econométricos consiste na utilização de **variáveis dummy (fictícia, postiça)**, frequentemente chamadas de variáveis **binárias ou dicotômicas**, uma vez que assumem apenas um de dois valores – em geral 0 ou 1 – para indicar a presença ou ausência de determinada característica



As **variáveis dummys** devem ser utilizadas sempre que desejarmos incluir **variáveis categóricas** em modelos que aceitam apenas **variáveis numéricas**



Em um caso de variável com **3 ou mais categorias**, seria necessário criar sempre **n-1 dummies**, pois o valor médio que esta categoria representa estará associada ao intercepto do modelo e  o peso das outras variáveis será considerada sempre em relação as observações da variável de intercepto


__------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------__


__------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------__

## $$Objetivo:$$   

## $$Criar ~um ~modelo~ de~ regressão~ linear~ com~variáveis~dummies$$

__------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------__


__------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------__

In [1]:
# Pacotes
import pandas as pd
import numpy as np

# Banco de dados 

In [2]:
#Carregar tabela
dados = pd.read_table("corrupcao.csv",sep=",",encoding="latin-1")
dados.head()


,pais,cpi,regiao
0,Argentina,3.9,América do Sul
1,Australia,8.7,Oceania
2,Austria,7.9,Europa
3,Belgium,7.1,Europa
4,Brazil,4.0,América do Sul


In [3]:
dados.shape

(50, 3)

**Tabela com 50 observações e 3 variáveis**

# $$Dicionário~ de~ Dados~ das~ variáveis$$

### **$$Variáveis:$$**

**pais:** Nome do País


**cpi:** Indice de percepção de corrupção dos habitante do País, Nota varia de 0 a 10, quanto maior a nota, melhor a percepção do mecanismo de corrupção


**regiao:** Continente que pertence o país 

# Analise Exploratória

In [4]:
#Transformar coluna em indice
dados = dados.set_index('pais')
dados.head()


,cpi,regiao
pais,,
Argentina,3.9,América do Sul
Australia,8.7,Oceania
Austria,7.9,Europa
Belgium,7.1,Europa
Brazil,4.0,América do Sul


**Coluna 'pais' é considerada um índice e não uma variável preditora**

In [5]:
#Observaçao das variaveis 
dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, Argentina to Iraq
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cpi     50 non-null     float64
 1   regiao  50 non-null     object 
dtypes: float64(1), object(1)
memory usage: 1.2+ KB


**Transformar a variável 'regiao' que é categórica em variáveis dummies, pois a regressão linear não modela variáveis categóricas**

In [6]:
#Criar variáveis dummies
dummies = pd.get_dummies(dados['regiao']);dummies.head(1)

,América do Sul,EUA e Canadá,Europa,Oceania,Ásia
pais,,,,,
Argentina,1,0,0,0,0


**Criar variáveis dummies para que uma variável categórica se transforme em uma numérica dicotômica, e assim conseguir criar um modelo de regressão linear**

In [7]:
#Drop variável dummy
dummies_drop = dummies.drop('América do Sul',axis=1)
dummies_drop.head()

,EUA e Canadá,Europa,Oceania,Ásia
pais,,,,
Argentina,0,0,0,0
Australia,0,0,1,0
Austria,0,1,0,0
Belgium,0,1,0,0
Brazil,0,0,0,0


**Observe que a categoria 'América do Sul' foi excluída do dataset**

**Na regressão linear devemos usar uma das categorias da variável qualitativa (Dummizadas) como categoria de referência (intercepto), e para que isso aconteça devemos excluir uma das categorias**

In [8]:
#Unindo os 2 datasets(variável target junto com as dummies)
dummies_drop['cpi'] = dados.cpi
dummies_drop.head()

,EUA e Canadá,Europa,Oceania,Ásia,cpi
pais,,,,,
Argentina,0,0,0,0,3.9
Australia,0,0,1,0,8.7
Austria,0,1,0,0,7.9
Belgium,0,1,0,0,7.1
Brazil,0,0,0,0,4.0


# Modelagem

In [9]:
#Selecionar as variáveis X e Y para a modelagem

x = dummies_drop[['EUA e Canadá','Europa','Oceania','Ásia']]

y = dummies_drop['cpi'].values


In [10]:
#Modelo
import statsmodels.api as sm

#Adicionar variável constante 
x1 = sm.add_constant(x)

#Criar modelo
modelo = sm.OLS(y,x1)
modelo_v1 = modelo.fit()
modelo_v1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     17.09
Date:                Tue, 12 Jul 2022   Prob (F-statistic):           1.37e-08
Time:                        14:50:34   Log-Likelihood:                -96.620
No. Observations:                  50   AIC:                             203.2
Df Residuals:                      45   BIC:                             212.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.1800      0.788      5.306      0.000       2.593       5.767
EUA e Canadá     3.8200      1.474      2.592      0.013       0.852       6.788
Europa           2.0783      0.866      2.400      0.021       0.334       3.822
Oceania          4.8200      1.474      3.271      0.002       1.852       7.788
Ásia            -1.8506      0.896     -2.065      0.045      -3.655      -0.046
==============================================================================
Omnibus:                        1.205   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.547   Jarque-Bera (JB):                1.083
Skew:                          -0.347   Prob(JB):                        0.582
Kurtosis:                       2.804   Cond. No.                         9.35
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Resultado da modelagem**

In [11]:
#Criando variáveis com os coeficientes para colocar na formula da regressão linear
intecepto = modelo_v1.params[0]
coef_eua_can = modelo_v1.params[1]
coef_europa = modelo_v1.params[2]
coef_oceania = modelo_v1.params[3]
coef_asia = modelo_v1.params[4]

#Coeficientes
modelo_v1.params

const           4.180000
EUA e Canadá    3.820000
Europa          2.078333
Oceania         4.820000
Ásia           -1.850588
dtype: float64

In [12]:
#Formula da regressão linear (Sem o uso de algoritimo)
dados['yhat'] = (intecepto + (coef_eua_can * x['EUA e Canadá'] + coef_europa * x['Europa']**2
                            + coef_oceania * x['Oceania']**3 + coef_asia * x['Ásia']**4))
dados.head(5)


,cpi,regiao,yhat
pais,,,
Argentina,3.9,América do Sul,4.180000
Australia,8.7,Oceania,9.000000
Austria,7.9,Europa,6.258333
Belgium,7.1,Europa,6.258333
Brazil,4.0,América do Sul,4.180000


**Previsão do modelo (yhat)**

In [13]:
#Interpretando os dados

#Tirando a média da variável target
dados3 = dados[['regiao','cpi']]
media_cpi=pd.DataFrame(dados3.groupby(['regiao']).mean())
media_cpi = media_cpi.iloc[1:,:]

#Rename
media_cpi .columns = ['Media_cpi']

#Adicionando os coeficientes
media_cpi['coeficientes'] = modelo_v1.params[1:]

#Adicionando o intercepto
media_cpi['intercepto'] = modelo_v1.params[0]
media_cpi

,Media_cpi,coeficientes,intercepto
regiao,,,
EUA e Canadá,8.000000,3.820000,4.18
Europa,6.258333,2.078333,4.18
Oceania,9.000000,4.820000,4.18
Ásia,2.329412,-1.850588,4.18


**Note que o valor das medias das regioes da variável target (cpi) é igual ao valor do intecepto + os coeficientes do modelo de regressão**

**OBS: A variável dummy 'América do Sul' é o intecepto, o valor do seu coeficiente é a média da cpi da sua região**

### Métricas de Avaliação

In [14]:
#Formula do R²

#Previsão
y_chapeu = dados['yhat']

#Media dos valores da variável target
y_media = dados['cpi'].mean()

#Valores da variável taget
y_real = dados['cpi']

#Formula do R²
r = sum((y_chapeu - y_media)**2)
r1 = sum((y_real -y_media )**2)
r2 = r/r1
print(f'R²= {r2*100:.1f}%')


R²= 60.3%


**O quanto as variáveis preditoras explicam a variável target**

In [15]:
# MAPE - Erro Percentual Absoluto Médio

y_real = dados['cpi']
y_chapeu= dados['yhat']
n = len(dados)
ab = abs(y_real - y_chapeu )/abs(y_real)
MAPE = sum(ab/n)
print(f'MAPE = {MAPE*100:.1f}% ')

MAPE = 37.3% 


**Métrica que mostra a porcentagem de erro em relação aos valores reais**

In [16]:
#MAE = Erro médio absoluto 

y_real = dados['cpi']
y_chapeu= dados['yhat']
n = len(dados)
ab = abs(y_real - y_chapeu )
MAE = sum(ab/n)
print(f'MAE = {MAE:.1f}, o modelo erra para cima ou para baixo sua previsao na unidade de medida da variavel resposta'  )

MAE = 1.3, o modelo erra para cima ou para baixo sua previsao na unidade de medida da variavel resposta


**Mede a média da diferença entre o valor real com o valor predito, esta métrica não é afetada por valores discrepantes — os denominados outliers**

In [17]:
#MSE = Erro quadrático médio, penaliza a metrica quando tem outliers

MSE = sum((y_chapeu - dados['cpi'])**2)
print(f'MSE = {MSE:.1f}, Soma dos erros ao quadrado')

MSE = 139.6, Soma dos erros ao quadrado


**Calcula a média de diferença entre o valor predito com o real ao quadrado**

In [18]:
#ERRO = Soma dos erros na regressao linear (Positivo e Negativo)

erro = sum(y_chapeu - dados['cpi'])
print(f'Erro= {erro:.1f}, diferença entre valor previsto e o valor real na linha de regressao')

Erro= -0.0, diferença entre valor previsto e o valor real na linha de regressao


**Soma dos erros do modelo em relação a linha de regressão sempre será 0 ou próxima de 0**

In [19]:
#MedianAE = Erro Mediano Absoluto

y_chapeu= dados['yhat']
y_mediana =dados['cpi'].median()
y_real = dados['cpi']
r = sum((y_chapeu - y_mediana)**2)
r1 = sum((y_real -y_mediana )**2)
MedianAE = r/r1;r2
print(f'MedianAE= {MedianAE*100:.1f}%')

MedianAE= 64.8%


**Igual o MAE - Erro médio absoluto, porém usando a mediana**

In [20]:
# RMSE = Raiz do erro quadrático médio , pois o erro medio é elevado ao quadrado para nao ter numeros negativos
import math

y_real = dados['cpi']
y_chapeu= dados['yhat']
n = len(dados)
e = (y_real - y_chapeu )**2
er = sum(e / n)
RMSE = math.sqrt(er)
print(f'RMSE= {RMSE:.2f}, quão distantes nossos valores previstos estão de nossos valores observados')


RMSE= 1.67, quão distantes nossos valores previstos estão de nossos valores observados


**No RMSE o erro retorna à unidade de medida do modelo**

# Conclusão

**A introdução das variáveis dummy na análise de regressão, constitui-se em importante instrumento
que amplia o poder de análise dos modelos, pois como o modelo de regressão não suporta variáveis categóricas, com as variáveis dummies é permitido a modelagem**
